Autor: Aguiar 

Ciência da Computação(Universidade Federal do Ceará - UFC) 

Mandacaru.Dev - B3 do Brasil

Descrição da atividade: Clusterizando Dados de Estatísticas Bancárias por Região Geográfica do Brasil

Objetivo: Implementar uma solução de clusterização para comparar cidades na mesma região geográfica

Baseado no Arquivo Base disponibilizado na plataforma Teams

In [ ]:
# Importações
import pandas as pd
import numpy as np

In [ ]:
# Armazena os dados em um DataFrame unico
df = pd.DataFrame()
for i in range(1,4): #Janeiro até Março (01 a 03)
  filename = "20210"+str(i)+"_ESTBAN_AG.CSV" # Geração do nome do arquivo
  dfAux = pd.read_csv(filename, sep=";",on_bad_lines='skip',encoding = "ISO-8859-1",  skiprows=2) # Para manipular o carregamento de CSV do ESTBAN
  df = pd.concat([df,dfAux], ignore_index=True) # Concatenação de dataframes

df.tail(10) 

In [ ]:
# Vamos analizar as colunas que ha no DataFrame
df.columns

In [ ]:
# Excluimos as linhas que possuem NaN
df = df.dropna(subset=['CODMUN'])
df[df['CODMUN'].isna()]

In [ ]:
# Vamos utilizar as colunas: 
#   - VERBETE_163_FIN_RURAIS_AGRICUL_CUST/INVEST, 
#   - VERBETE_164_FIN_RURAIS_PECUAR_CUST/INVEST, 
#   - VERBETE_165_FIN_RURAIS_AGRICUL_COMERCIALIZ, 
#   - VERBETE_166_FIN_RURAIS_PECUARIA_COMERCIALIZ, 
#   - VERBETE_169_FINANCIAMENTOS_IMOBILIARIOS
# Que correspondem às colunas 23-26 e 28 do nosso dataframe:

cols = df.columns
selectedCols = []
# selecionando colunas 0-6, 20, 28 e última coluna do dataframe
selectedCols = list(cols[0:7]) + list(cols[23:27]) + [cols[28],cols[-1]]
selectedCols


In [ ]:
# Deixaremos apenas as colunas selecioandas no nosso DataFrame, ou seja vamos apagar do nosso atual dataFrame as colunas que são diferentes que as da lista selecionada
df = df.drop(df.columns.difference(selectedCols), axis=1)
df.head()

In [ ]:
# Analizamos se os tipos de dados estão de acordo com o que precisamos que eles estejam, por exemplo código deve esta em String para nossa analize
df.dtypes
# note que o codigo de IBGE deve ser String e esta em FLOAT
df = df.astype({'CODMUN_IBGE': 'int64'})
df = df.astype({'CODMUN_IBGE': 'str'})
df.dtypes
df.head()

In [ ]:
# Utilizaremos dados somente da regiao centro-oeste é formada por 4 unidades de federação, sendo:
#  Distrito Federal – Brasília (DF)
#  Goiás – Goiânia (GO)
#  Mato Grosso – Cuiabá (MT)
#  Mato Grosso do Sul – Campo Grande (MS)

# criamos o dataFrame que recebe os dados corespondentes aos estados do Centro-Oeste
dt_co = pd.DataFrame()
dt_co = df[df['UF'] == "DF"]
dt_co = dt_co.append(df[df['UF'] == "GO"])
dt_co = dt_co.append(df[df['UF'] == "MT"])
dt_co = dt_co.append(df[df['UF'] == "MS"])

dt_co # dataFrame com dados das cidades do Centro-Oeste

In [ ]:
# Selecione as colunas que queremos trabalhar
selectedCols = dt_co.columns[7:-2]
selectedCols

Index(['VERBETE_163_FIN_RURAIS_AGRICUL_CUST/INVEST',
       'VERBETE_164_FIN_RURAIS_PECUAR_CUST/INVEST',
       'VERBETE_165_FIN_RURAIS_AGRICUL_COMERCIALIZ',
       'VERBETE_166_FIN_RURAIS_PECUARIA_COMERCIALIZ'],
      dtype='object')

In [ ]:
# Somamos as colunas e adicionamos a nova coluna soma_fin_rurais
dt_co['soma_fin_rurais'] = dt_co[selectedCols].sum(axis=1)

# Vamos definir a coluna CODMUN_IBGE como indice prq receberemos os dados dos municipios e o indice deles eh o codigo do municipio
dt_co.set_index("CODMUN_IBGE", inplace = True)
dt_co.head()

,#DATA_BASE,UF,CODMUN,MUNICIPIO,CNPJ,NOME_INSTITUICAO,AGENCIA,VERBETE_163_FIN_RURAIS_AGRICUL_CUST/INVEST,VERBETE_164_FIN_RURAIS_PECUAR_CUST/INVEST,VERBETE_165_FIN_RURAIS_AGRICUL_COMERCIALIZ,VERBETE_166_FIN_RURAIS_PECUARIA_COMERCIALIZ,VERBETE_169_FINANCIAMENTOS_IMOBILIARIOS,soma_fin_rurais
CODMUN_IBGE,,,,,,,,,,,,,
5300108,202101,DF,35800.0,BRASILIA,0,BCO DO BRASIL S.A.,'00000000045292,0,0,0,0,0,0
5300108,202101,DF,35800.0,BRASILIA,0,BCO DO BRASIL S.A.,'00000000083461,100966,0,0,0,35230965,100966
5300108,202101,DF,35800.0,BRASILIA,0,BCO DO BRASIL S.A.,'00000000083542,288547,0,0,0,30531871,288547
5300108,202101,DF,35800.0,BRASILIA,0,BCO DO BRASIL S.A.,'00000000089400,0,0,0,0,94928,0
5300108,202101,DF,35800.0,BRASILIA,0,BCO DO BRASIL S.A.,'00000000158496,191564,0,0,0,54876843,191564


In [ ]:
# proximo passo eh conseguir os dados do municipios
# tratar eles de maneira semelhante aos dados acima tratados

# dividir a coluna da soma com a da populacao e conseguir a coluna perCapita 'fin_agro_per_capita'mbmmb

In [ ]:
# Obtendo os dados populacionais - Todas as regiões
import requests                    # api module
import json
url = "https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2021/variaveis/9324?localidades=N6[N2[1,2,3,4,5]]"
response = requests.get(url)
population = response.json()
for item in population:
  for key in item['resultados']:
    populationJson = key
populationJson.pop('classificacoes')
dfPopulation = pd.json_normalize(populationJson['series'])
dfPopulation.rename(columns = {dfPopulation.columns[0]:'CODMUN_IBGE', 
                         dfPopulation.columns[-1]: 'POPULACAO'},
             inplace = True)
dfPopulation[['CIDADE','UF']] = dfPopulation[dfPopulation.columns[-2]].str.split(' - ',1).tolist()
dfPopulation.set_index('CODMUN_IBGE', inplace = True)
dfPopulation.drop(dfPopulation.columns[0:3], axis=1,inplace=True)
dfPopulation = dfPopulation.astype({'POPULACAO': 'int64'})
dfPopulation.head()



# essa requisicao colhe os dados aglomerados de 2010, por UF(as Uf's do Centro-Oeste), populacao total, com numero dos municipios, populacao nos municipios e agregado numero 1290
url = "https://servicodados.ibge.gov.br/api/v3/agregados/1290/periodos/2010/variaveis/603|1000603|609|1000609?localidades=N3[50,51,52,53]&classificacao=11275[91176]"
response = requests.get(url)
population = response.json() 

# agora que temos o Json de resposta, vamos diminuir os níveis
for item in population:
  for key in item['resultados']:
    populationJson = key
populationJson.pop('classificacoes')

# Vamos passar o Json pelo pandas usando o metodo json_normalize
dfPopulation = pd.json_normalize(populationJson['series'])

In [ ]:


# essa requisicao colhe os dados aglomerados de 2010, por UF(as Uf's do Centro-Oeste), populacao total, com numero dos municipios, populacao nos municipios e agregado numero 1290
url = "https://servicodados.ibge.gov.br/api/v3/agregados/793/periodos/2007/variaveis/93?localidades=N3[50,51,52,53]"
response = requests.get(url)
population = response.json() 

# agora que temos o Json de resposta, vamos diminuir os níveis
for item in population:
  for key in item['resultados']:
    populationJson = key
populationJson.pop('classificacoes')

# Vamos passar o Json pelo pandas usando o metodo json_normalize
dfPopulation = pd.json_normalize(populationJson['series'])
dfPopulation.head()

,localidade.id,localidade.nivel.id,localidade.nivel.nome,localidade.nome,serie.2007
0,50,N3,Unidade da Federação,Mato Grosso do Sul,2265274
1,51,N3,Unidade da Federação,Mato Grosso,2854642
2,52,N3,Unidade da Federação,Goiás,5647035
3,53,N3,Unidade da Federação,Distrito Federal,2455903


In [ ]:
# Obtendo os dados populacionais - Todas as regiões
#import requests                    # api module
#import json

#population = pd.read_csv('tabela6579.csv')

#population.head()

import pandas as pd
#df = pd.read_csv('tabela6579.csv')

city = pd.DataFrame()
city = pd.read_csv("tabela6579AllCitys.csv", sep=";",on_bad_lines='skip',encoding = "ISO-8859-1",  skiprows=2) 
#dfAux = pd.read_csv("tabela6579AllCitys.csv", sep=";",on_bad_lines='skip',encoding = "ISO-8859-1",  skiprows=2) 
#df = pd.concat([df,dfAux], ignore_index=True) # Concatenação de dataframes

city.tail(10) 

,NÃ­vel,CÃ³d.,MunicÃ­pio,Ano
576,29 - PopulaÃ§Ã£o judicial do municÃ­pio de Ita...,NaN,NaN,NaN
577,NaN,NaN,NaN,NaN
578,Legenda,NaN,NaN,NaN
579,SÃ­mbolo,Significado,NaN,NaN
580,-,"Zero absoluto, nÃ£o resultante de um cÃ¡lculo ...",NaN,NaN
581,0,Zero resultante de um cÃ¡lculo ou arredondamen...,NaN,NaN
582,X,Valor inibido para nÃ£o identificar o informan...,NaN,NaN
583,..,Valor nÃ£o se aplica.\nEx: NÃ£o se pode obter ...,NaN,NaN
584,...,Valor nÃ£o disponÃ­vel.\nEx: A produÃ§Ã£o de f...,NaN,NaN
585,A a Z\n(exceto X),Significa uma faixa de valores. Varia em funÃ§...,NaN,NaN
